# Coding your own RNN

Using this pre-filled notebook, we will code our own RNN for sentence classification. For now, we'll keep using IMDB, as the goal of this part is to understand how an RNN works.

Unlike our previous lab, we will also learn the embedding layer. Which means we need to deal with vocabulary by ourselves.

In [ ]:
!pip install datasets torch numpy scikit-learn torchtext tqdm

In [4]:
from functools import partial
from typing import Callable, Dict, Generator, List, Tuple

from datasets import load_dataset
import numpy as np
from sklearn.utils import shuffle
import torch
from torch import nn
from torchtext.vocab import vocab, Vocab
from torchtext.data.utils import get_tokenizer

from tqdm.auto import tqdm

## Dataset
We load the dataset and split the training set in a stratified train/validation set.

In [5]:
dataset = load_dataset("imdb")
train_dataset = dataset["train"].train_test_split(
    stratify_by_column="label", test_size=0.2, seed=42
)
test_df = dataset["test"]
train_df = train_dataset["train"]
valid_df = train_dataset["test"]
train_df.shape, valid_df.shape, test_df.shape

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

((20000, 2), (5000, 2), (25000, 2))

## Vocabulary (1 point)

**\[1 point\]** Build your own vocabulary. The [example provided in torchtext documentation](https://pytorch.org/text/stable/vocab.html#id1) might be of help.
* Don't forge to setup the `min_freq` parameter to not include unfrequent noise.
* You will need a tokenizer. Reuse the `basic_english` one from the our previous lab.
* For an RNN we need two special tokens: `<unk>`, for unknown words, and `<pad>` for padding.

In [11]:
from collections import Counter, OrderedDict
tokenizer = get_tokenizer("basic_english", language="en")

words = [word for sentence in train_df['text'] for word in tokenizer(sentence)]
counter = Counter(words)
sorted_by_freq_tuples = sorted(counter.items(), key=lambda x: x[1], reverse=True)
ordered_dict = OrderedDict(sorted_by_freq_tuples)
unk_token = '<unk>'
pad_token = '<pad>'
vocabulary = vocab(ordered_dict=ordered_dict, specials=[unk_token, pad_token], min_freq=3)
vocabulary.set_default_index(vocabulary[unk_token])

## Vectorize and batch the input (3 points)

As seen in class, our model should take one-hot encoded vectors corresponding to the each token vocabulary id. However, computing a vector x matrix multiplication for every input is unnecessarily costly. Multiplying a one-hot vector with a matrix is the equivalent of taking one row of the matrix. In pyTorch, we provide ids for each token which will be used as input to an `nn.Embedding` layer. The id is simply the row in the embedding matrix.

**\[1 point\]** Fill the `vectorize_text` function returning a 1D torch tensor of `torch.long` for each input text.

In [15]:
def vectorize_text(
    text: str, vocabulary: Vocab, tokenizer: Callable[[str], List[str]]
) -> torch.Tensor:
    """
    Generate a tensor of vocabluary IDs for a given text.
    Args:
        text: the input text.
        vocabulary: a Vocab objects.
        tokenizer: a text tokenizer.
    Returns:
        A tensor of IDs (torch.long).
    """
    tokens = tokenizer(text)
    return torch.as_tensor([vocabulary[token] for token in tokens])

In [16]:
text_pipeline = partial(vectorize_text, vocabulary=vocabulary, tokenizer=tokenizer)

Check the function is working correctly, especially it should return the right special id for unknown words.

In [17]:
text_pipeline("Some text I am thinking about... ragafqfa")

tensor([  56, 3160,   13,  244,  526,   50,    3,    3,    3,    0])

In [18]:
X_train = [text_pipeline(text) for text in tqdm(train_df["text"])]
y_train = train_df["label"]
X_valid = [text_pipeline(text) for text in tqdm(valid_df["text"])]
y_valid = valid_df["label"]
X_test = [text_pipeline(text) for text in tqdm(test_df["text"])]
y_test = test_df["label"]

  0%|          | 0/20000 [00:00<?, ?it/s]

  0%|          | 0/5000 [00:00<?, ?it/s]

  0%|          | 0/25000 [00:00<?, ?it/s]

To speed up the training process, we turn the inputs into batches, as we did last time. For batches to work, every line must have the same lengths. Last time, it was implicit as only a vector (the average of all embeddings) was provided. This time, every line has the length of a different review.

To go around this problem, we use padding. So every line within a batch is padded to the length of its longest element.

* **\[1 point\]** Fill the data generator function.
* **\[1 point\]** On which side should you pad and why?

In [19]:
def data_generator(
    X: List[torch.tensor], y: List[int], pad_id: int, batch_size: int = 32
) -> Generator[Tuple[torch.Tensor, torch.Tensor], None, None]:
    """
    Yield batches from given input data and labels.
    Args:
        X: a list of tensor (input features).
        y: the corresponding labels.
        batch_size: the size of every batch [32].
    Returns:
        A tuple of tensors (features, labels).
    """
    X, y = shuffle(X, y)
    for i in range(0, len(X) - batch_size, batch_size):
        max_length = 0
        for j in range(i, i + batch_size):
            if X[j].size()[0] > max_length:
              max_length = X[j].size()[0]
        for j in range(i, i + batch_size):
            pad = max_length - X[j].size()[0]
            pad_tensor = torch.full((pad,), pad_id)
            X[j] = torch.cat((pad_tensor, X[j]))

        yield (torch.stack(X[i:i+batch_size]), torch.tensor(y[i:i+batch_size], dtype=torch.long))

In [20]:
train_gen = lambda: data_generator(X_train, y_train, vocabulary[pad_token])
valid_gen = lambda: data_generator(X_valid, y_valid, vocabulary[pad_token])
test_gen = lambda: data_generator(X_test, y_test, vocabulary[pad_token])

In [46]:
for batch in data_generator(X_train, y_train, vocabulary[pad_token]):
  print(batch[0].size())
  break

torch.Size([32, 674])


## Classifier (3 points)

**\[3 points\]** Code your own RNN. Fill the `RNN` class correctly. Remember an RNN has 3 matrices and an embedding layer (see course slide 61).
* The embedding layer turns a one-hot vectors into dense vectors.
* The first matrix (W) connects the embedding to the hidden layer.
  * `embedding_size -> hidden_size`
* The second matrix (U) connect the previous hidden layer to the current one.
  * `hidden_size -> hidden_size`
* These to vectors are added and go through an activation function (e.g. $h_t = tanh(Wx_i+Uh_{t-1})$).
* The last matrix (V) connects the hidden layer to the hidden layer to the output.
  * `hidden_size -> 1`
* Donc forget to add an `init_hidden` function which initialize the first hidden layer to 0.

In [47]:
import torch
import torch.nn as nn
from typing import Tuple

class RNN(nn.Module):
    """A basic RNN implementation"""

    def __init__(self, vocab_size: int, nb_embedding: int, nb_hidden: int) -> None:
        """
        Args:
            vocab_size: the number of words in your vocabulary.
            nb_embedding: the dimension of the embedding vector.
            nb_hidden: the dimension of the hidden state.
        """
        super().__init__()
        self.embedded = nn.Embedding(vocab_size, nb_embedding)
        self.W = nn.Linear(nb_embedding, nb_hidden)
        self.U = nn.Linear(nb_hidden, nb_hidden)
        self.output = nn.Linear(nb_hidden, 1)
        self.nb_hidden = nb_hidden

    def init_hidden(self, batch_size):
        # Initialize hidden state with zeros
        hidden = torch.zeros(batch_size, self.nb_hidden)
        return hidden

    def forward(self, x: torch.Tensor, h: torch.Tensor) -> Tuple[torch.Tensor, torch.Tensor]:
        """
        Args:
            x: the input tensor (token indices).
            h: the hidden tensor from the previous time step.
        Returns:
            The output of the RNN layer and the new hidden state.
        """
        embedding = self.embedded(x)
        W = self.W(embedding)
        U = self.U(h)
        hidden_state = torch.tanh(W + U)
        output = self.output(hidden_state)
        return output, hidden_state


## Training (2 points)

Training is a bit different than usual. We will need to sequentially (but in "batch parallel") go through an input, keeping track of the hidden layer, and use the last output as prediction.

**\[2 point\]** Code the training loop.
* Note that for each batch, you need to loop through the whole input and use the output of the last token as input to your criterion.
* Keep the best model evaluated on the validation set.
* Plot the training and validation losses.
* Training will take some time (~30 min on a T4 GPU). Make sure your results appear in the notebook.

In [109]:
import matplotlib.pyplot as plt

def train(model, criterion, optimizer, train_loader, valid_loader, epochs, device):
  best_valid_loss = float('inf')
  train_losses, valid_losses = [], []

  for epoch in tqdm(range(epochs)):
      model.train()
      total_train_loss = 0

      for x_batch, y_batch in tqdm(train_gen()):
        x_batch = x_batch.to(device)
        y_batch = y_batch.to(device)
        h = model.init_hidden(x_batch.size(0)).to(device)

        for t in range(x_batch.size(1)):  # Loop through each token in the sequence
          outputs, h = model(x_batch[:,t], h)

        loss = criterion(outputs[:,-1], y_batch.float())
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        total_train_loss += loss.item()

      train_losses.append(total_train_loss / len(X_train))

      model.eval()
      total_valid_loss = 0

      with torch.no_grad():
          for x_batch, y_batch in valid_gen():
              x_batch = x_batch.to(device)
              y_batch = y_batch.to(device)
              h = model.init_hidden(x_batch.size(0)).to(device)

              for t in range(x_batch.size(0)):
                  outputs, h = model(x_batch[:,t], h)

              loss = criterion(outputs[:,-1], y_batch.float())
              total_valid_loss += loss.item()

      valid_losses.append(total_valid_loss / len(X_valid))

      if total_valid_loss < best_valid_loss:
          best_valid_loss = total_valid_loss
          torch.save(model.state_dict(), 'best_model.pt')

      print(f'Epoch: {epoch+1:02}, Train Loss: {train_losses[-1]:.3f}, Val. Loss: {valid_losses[-1]:.3f}')

  # Plot training and validation loss
  plt.plot(train_losses, label='Train Loss')
  plt.plot(valid_losses, label='Validation Loss')
  plt.xlabel('Epoch')
  plt.ylabel('Loss')
  plt.legend()
  plt.show()

In [107]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"
device

'cuda:0'

In [110]:
n_embedding = 32
n_hidden = 64
model = RNN(len(vocabulary.get_itos()), n_embedding, n_hidden).to(device)
criterion = nn.BCEWithLogitsLoss()
optimizer = torch.optim.RMSprop(model.parameters(), lr=0.001)
train(model, criterion, optimizer, train_gen, valid_gen, 20, device)

  0%|          | 0/20 [00:00<?, ?it/s]

0it [00:00, ?it/s]

Epoch: 01, Train Loss: 0.021, Val. Loss: 0.022


0it [00:00, ?it/s]

Epoch: 02, Train Loss: 0.020, Val. Loss: 0.026


0it [00:00, ?it/s]

Epoch: 03, Train Loss: 0.019, Val. Loss: 0.022


0it [00:00, ?it/s]

KeyboardInterrupt: ignored

## Evaluation (1 point)

* **\[1 point\]** Compute the accuracy for all 3 splits.

In [113]:
# Your code
# Note that we need to use a sigmoid on the output now.
model.load_state_dict(torch.load('best_model.pt'))
def evaluate(model: nn.Module, dataloader: Generator[Tuple[torch.Tensor, torch.Tensor], None, None]) -> float:
    """
    Function evaluating the model on dataset.
    """
    correct = 0
    total = 0
    with torch.no_grad():
       for x_batch, y_batch in valid_gen():
              x_batch = x_batch.to(device)
              y_batch = y_batch.to(device)
              h = model.init_hidden(x_batch.size(0)).to(device)

              for t in range(x_batch.size(0)):
                  outputs, h = model(x_batch[:,t], h)

              predicted = torch.round(torch.sigmoid(outputs[:,-1]))
              total += y_batch.size(0)
              correct += (predicted == y_batch).sum().item()
    return correct / total
def accuracy(model: nn.Module) -> None:
    """
    Print the accuracy of the model for the 3 dataset (train, val, test) ordered
    """
    with torch.no_grad():
        train_accuracy = evaluate(model, train_gen())
        val_accuracy = evaluate(model, valid_gen())
        test_accuracy = evaluate(model, test_gen())
    print(train_accuracy, val_accuracy, test_accuracy)

accuracy(model)

0.5 0.5006009615384616 0.4973958333333333
